In [23]:
import pandas as pd
import os
import pickle
import copy

from rdkit import Chem
from ConformationEnsemble import ConformationEnsembleLibrary
from ConfEnsembleDataset import ConfEnsembleDataset
from ccdc.conformer import ConformerGenerator
from ccdc.molecule import Molecule
from tqdm import tqdm

In [ ]:
# To be able to save conformer properties
Chem.SetDefaultPickleProperties(Chem.PropertyPickleOptions.AllProps) 

In [2]:
data_dir_path = 'data/'
pdbbind_refined_dir_path = '../PDBBind/PDBbind_v2020_refined/refined-set/'
pdbbind_general_dir_path = '../PDBBind/PDBbind_v2020_other_PL/v2020-other-PL/'

In [3]:
widths = [6,6,7,6,17,9,200]
cols = 'PDB code, resolution, release year, -logKd/Ki, Kd/Ki, reference, ligand name'.replace(', ', ',').split(',')
pl_data = pd.read_fwf(os.path.join(pdbbind_refined_dir_path, 'index', 'INDEX_general_PL_data.2020'), widths=widths, skiprows=6, header=None)
pl_data.columns=cols

In [4]:
pl_data.head()

,PDB code,resolution,release year,-logKd/Ki,Kd/Ki,reference,ligand name
0,3zzf,2.20,2012,0.40,Ki=400mM //,3zzf.pdf,(NLG)
1,3gww,2.46,2009,0.45,IC50=355mM //,3gwu.pdf,(SFX)
2,1w8l,1.80,2004,0.49,Ki=320mM //,1w8l.pdf,(1P3)
3,3fqa,2.35,2009,0.49,IC50=320mM //,3fq7.pdf,(GAB&PMP)
4,1zsb,2.00,1996,0.60,Kd=250mM //,1zsb.pdf,(AZM)


In [5]:
pl_data = pl_data[~pl_data['ligand name'].str.contains('-mer')]

In [6]:
pl_data.shape

(16844, 7)

In [7]:
correct_pdb_ids = pl_data['PDB code'].values
#pdb_ids = sorted([dir for dir in os.listdir(pdbbind_dir) if len(dir) == 4])

In [8]:
def extract_pdbbind_mols(directory_path, query_pdb_ids) :
    mols = []
    pdb_ids = [pdb_id for pdb_id in os.listdir(directory_path) if pdb_id in query_pdb_ids]
    for pdb_id in pdb_ids :
        mol2path = os.path.join(directory_path, pdb_id, f'{pdb_id}_ligand.mol2')
        try :
            mol = Chem.rdmolfiles.MolFromMol2File(mol2path, removeHs=False)
            if mol is not None :
                rdmol = Chem.MolFromSmiles(Chem.MolToSmiles(mol))
                if rdmol is not None : #rdkit parsable
                    #mol = PropertyMol(mol)
                    mol.GetConformer().SetProp('PDB_ID', pdb_id)
                    mols.append(mol)
                else :
                    print('Not RDKit parsable')
        except :
            print('Impossible to read mol2 file for ' + pdb_id)
            
    return mols

In [9]:
general_mols = extract_pdbbind_mols(pdbbind_general_dir_path, correct_pdb_ids)
print(len(general_mols))

RDKit WARNING: [13:42:41] 6a05_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:41] 6a05_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:41] 1r0x_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:41] 1r0x_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:41] 1w1g_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:41] 1w1g_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:41] 2k0g_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:41] 1it6_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:41] 4lph_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:41] 6oh3_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:41] 3fjz_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:41] 6eo0_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING

Not RDKit parsable


RDKit WARNING: [13:42:42] 2ork_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:42] 4rad_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:42] 6dd1_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:42] 4s3f_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:42] 5y62_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:42] 2ork_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:42] 4m7y_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:42] 4rad_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:42] 6dd1_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:42] 4s3f_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:42] 5y62_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:42] 4m7y_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:42] 2e9c_ligand: warning - O.co2 wit

Not RDKit parsable


RDKit WARNING: [13:42:43] sanitise [13:42:43] 4i60_ligand: [13:42:43] 4lq3_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:43] sanitise [13:42:43] 4i60_ligand: [13:42:43] 4lq3_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:43] 1njf_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:43] 6cv8_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:43] 1njf_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:43] 5kqd_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:43] 4b6u_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:43] 2qpj_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:43] 6cv8_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:43] 5kqd_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:43] 4b6u_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:43] 2qpj_ligand: warning - O.co2 w

...

RDKit WARNING: [13:42:48] 1akv_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:48] 1akv_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:48] 1iig_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:48] 1iig_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:48] 5jbi_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:48] 5jbi_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:48] 3kr5_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:48] 3kr5_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:48] 5vjn_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:48] 5vjn_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:48] 3c14_ligand: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[13:42:48] 3c14_ligand: Warning - no explicit hydrogens in mol2 file but needed for formal

Not RDKit parsable


RDKit WARNING: [13:42:49] 6apv_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:49] 6apv_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:49] 5wrs_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:49] 1xxi_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:49] 5afx_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:49] 5wrs_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:49] 1xxi_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:49] 5afx_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:49] 4nyt_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:49] 5l0c_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:49] 4nyt_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:49] 5l0c_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:49] 5jv2_ligand: warning - O.co2 wit

10634


RDKit ERROR: [13:42:50] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6
RDKit ERROR: 
RDKit WARNING: [13:42:50] sanitise [13:42:50] 1hyz_ligand: [13:42:50] 5foo_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:50] 1q6t_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:50] 2pu0_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:50] 1a50_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:50] 1w1d_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:50] 6n94_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:50] 6giu_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:50] 2wtx_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:50] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6

[13:42:50] sanitise [13:42:50] 1hyz_ligand: [13:42:50] 5foo_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:50] 2wtx_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:

In [10]:
refined_mols = extract_pdbbind_mols(pdbbind_refined_dir_path, correct_pdb_ids)
print(len(refined_mols))

RDKit WARNING: [13:42:51] 1pdz_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:51] 1pdz_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:51] 5cks_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:51] 2zz2_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:51] 5cks_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:51] 3fzn_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:51] 2e94_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:51] 4u6c_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:51] 2zz2_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:51] 3fzn_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:51] 2e94_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:51] 4u6c_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:51] 1ado_ligand: warning - O.co2 wit

4400


RDKit WARNING: [13:42:54] 1dar_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:54] 1dar_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:54] 1wur_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:54] 3d7k_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:54] 3ocp_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:54] 2zcq_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:54] 5n3v_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:54] 2c92_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [13:42:54] 4q46_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:54] 1wur_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:54] 3d7k_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:54] 3ocp_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[13:42:54] 2zcq_ligand: warn

In [11]:
with open(os.path.join(data_dir_path, 'pdbbind_general_mol_list_with_h.p'), 'wb') as f :
    pickle.dump(general_mols, f)

In [12]:
# with open(os.path.join(data_dir_path, 'pdbbind_general_mol_list_with_h.p'), 'rb') as f :
#     general_mols = pickle.load(f)

In [13]:
with open(os.path.join(data_dir_path, 'pdbbind_refined_mol_list_with_h.p'), 'wb') as f :
    pickle.dump(refined_mols, f)

In [14]:
# with open(os.path.join(data_dir_path, 'pdbbind_refined_mol_list_with_h.p'), 'rb') as f :
#     refined_mols = pickle.load(f)

In [15]:
general_CEL = ConformationEnsembleLibrary(general_mols)
refined_CEL = ConformationEnsembleLibrary(refined_mols)

100%|█████████████████████████████████████| 4400/4400 [00:00<00:00, 7962.50it/s]


In [16]:
os.makedirs(os.path.join(data_dir_path, 'raw'), exist_ok=True)

In [17]:
with open(os.path.join(data_dir_path, 'raw', 'pdbbind_general_conf_ensemble_library_with_h.p'), 'wb') as f :
    pickle.dump(general_CEL, f)

In [19]:
with open(os.path.join(data_dir_path, 'raw', 'pdbbind_refined_conf_ensemble_library_with_h.p'), 'wb') as f :
    pickle.dump(refined_CEL, f)

In [40]:
with open(os.path.join(data_dir_path, 'raw', 'pdbbind_general_conf_ensemble_library_with_h.p'), 'rb') as f :
    general_CEL = pickle.load(f)

In [41]:
with open(os.path.join(data_dir_path, 'raw', 'pdbbind_refined_conf_ensemble_library_with_h.p'), 'rb') as f :
    refined_CEL = pickle.load(f)

In [2]:
# %%time
# general_dataset = ConfEnsembleDataset('data/')
# refined_dataset = ConfEnsembleDataset('data/', split='refined')

# Datasets are now ready to use

INFO: /home/benoit/CCDC/CSD_2021/bin/mogul
INFO: /home/benoit/CCDC/CSD_2021/bin/mogul
INFO: /home/benoit/CCDC/CSD_2021/bin/mogul
INFO: /home/benoit/CCDC/CSD_2021/bin/mogul
INFO: /home/benoit/CCDC/CSD_2021/bin/mogul
INFO: /home/benoit/CCDC/CSD_2021/bin/mogul
INFO: /home/benoit/CCDC/CSD_2021/bin/mogul
INFO: /home/benoit/CCDC/CSD_2021/bin/mogul
INFO: /home/benoit/CCDC/CSD_2021/bin/mogul
CPU times: user 957 ms, sys: 1.73 s, total: 2.69 s
Wall time: 2.82 s
